In [137]:
%cd "C:/Users/walee/OneDrive/Desktop/jarvis/research"

C:\Users\walee\OneDrive\Desktop\jarvis\research


In [138]:
%pwd

'C:\\Users\\walee\\OneDrive\\Desktop\\jarvis\\research'

In [139]:
import os
os.chdir("../")

In [140]:
%pwd


'C:\\Users\\walee\\OneDrive\\Desktop\\jarvis'

In [141]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [142]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [143]:
extracted_data=load_pdf_file(data='Data/')

incorrect startxref pointer(1)
parsing for Object Streams
found 0 objects within Object(280,0) whereas 100 expected
found 0 objects within Object(281,0) whereas 100 expected
found 0 objects within Object(282,0) whereas 100 expected
found 0 objects within Object(283,0) whereas 100 expected
found 0 objects within Object(284,0) whereas 100 expected
found 0 objects within Object(285,0) whereas 100 expected
found 0 objects within Object(286,0) whereas 100 expected
found 0 objects within Object(287,0) whereas 100 expected
found 0 objects within Object(288,0) whereas 100 expected
found 0 objects within Object(289,0) whereas 100 expected
found 0 objects within Object(290,0) whereas 100 expected
found 0 objects within Object(291,0) whereas 100 expected
found 0 objects within Object(292,0) whereas 100 expected
found 0 objects within Object(293,0) whereas 100 expected
found 0 objects within Object(294,0) whereas 100 expected
found 0 objects within Object(295,0) whereas 100 expected
found 0 object

In [144]:
# extracted_data

In [145]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [146]:
text_chunks = text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 675


In [147]:
# text_chunks

In [148]:
from langchain.embeddings import HuggingFaceEmbeddings

In [149]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [150]:
embeddings = download_hugging_face_embeddings()

In [151]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [152]:
# query_result

In [153]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [154]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [155]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "payrollbot"

pc.create_index(
    name=index_name,
    dimension=384,  # Dimension of the embeddings
    metric="cosine",  # Similarity metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '634e43dedd3a5519094726dc6a3a3c48', 'date': 'Tue, 23 Sep 2025 02:22:39 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [156]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [157]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [158]:
# Load Existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [159]:
docsearch

In [160]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [168]:
retrieved_docs = retriever.invoke("What is FUTA?")

c:\Users\walee\anaconda3\envs\FRAI\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [169]:
retrieved_docs

[Document(id='486d6ba1-7d9c-4c39-9f80-fa6b86e365ad', metadata={'author': 'W:CAR:MP:FP', 'creationdate': '2024-12-17T11:23:27-05:00', 'creator': 'AH XSL Formatter V6.6 MR4 for Linux64 : 6.6.6.37929 (2019/03/11 12:35JST)', 'moddate': '2024-12-18T09:38:20-05:00', 'page': 47.0, 'page_label': '48', 'producer': 'Antenna House PDF Output Library 6.6.1437 (Linux64); modified using iText 2.1.7 by 1T3XT', 'source': 'Data\\p15.pdf', 'subject': "(Circular E), Employer's Tax Guide", 'title': '2025 Publication 15', 'total_pages': 58.0, 'trapped': '/False'}, page_content='FUTA\nFringe benefits. T axable on excess of fair market value of the benefit over the sum of an amount paid for it \nby the employee and any amount excludable by law. However, special valuation rules may \napply. Benefits provided under cafeteria plans may qualify for exclusion from wages for \nsocial security, Medicare, and FUTA taxes. See Pub. 15-B for details.\nGovernment employment:\n1. State/local governments and political \ns

In [163]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks for payroll. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [165]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "What is FUTA tax?"})
print(response["answer"])

c:\Users\walee\anaconda3\envs\FRAI\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)




I am an AI assistant for payroll tasks. I do not have a name.
